In [147]:
import optuna as optuna
from stable_baselines3 import PPO
from pykalman import KalmanFilter
import gym
import numpy as np
import pandas as pd
from gym import spaces
from scipy.signal import periodogram
from scipy.stats import norm
from stable_baselines3.common.vec_env import DummyVecEnv

In [148]:
class StockTradingEnv(gym.Env):
    def __init__(self, data, window_size=10, initial_balance=10000, n_envs=1):
        super(StockTradingEnv, self).__init__()

        # Initialize instance variables
        self.data = data
        self.window_size = window_size
        self.initial_balance = initial_balance
        self.positions = {}
        self.current_step = self.window_size
        self.done = False

        # Extract tickers from data
        self.tickers = list(set(data['Ticker']))

        # Set action and observation spaces
        self.action_space = spaces.Box(low=-1, high=1, shape=(len(self.tickers),), dtype=np.float32)
        #TODO derive the 33451 number from the variables at play in next_observation
        self.observation_space = spaces.Box(low=0, high=1, shape=(1, 33451), dtype=np.float32)

        # Set number of environments to 1
        self.n_envs = n_envs

        # Set up the observation buffer
        self._obs_buffer = np.zeros((self.n_envs,) + self.observation_space.shape, dtype=self.observation_space.dtype)

        # Reset the environment
        self.reset()

    def reset(self):
        self.balance = self.initial_balance
        self.current_step = self.window_size
        self.done = False
        self.positions = {}

        obs = np.empty((self.n_envs,) + self.observation_space.shape, dtype=self.observation_space.dtype)
        for i in range(self.n_envs):
            obs[i] = self._next_observation()

        return obs

    def step(self, action):
        current_prices = self.data.iloc[self.current_step][['Close', 'Ticker']].values
        current_prices = np.expand_dims(current_prices, axis=0)
        current_prices = np.tile(current_prices, (len(self.tickers), 1))
        actions = action * self.balance

        for i, t in enumerate(self.tickers):
            if actions[i] > 0:
                shares_to_buy = int(actions[i] // current_prices[i, 0])
                cost = shares_to_buy * current_prices[i, 0]
                if cost > self.balance:
                    shares_to_buy = 0
                    cost = 0
                else:
                    self.balance -= cost
                if t not in self.positions:
                    self.positions[t] = 0
                self.positions[t] += shares_to_buy
            elif actions[i] < 0:
                shares_to_sell = int(min(self.positions.get(t, 0), np.abs(actions[i]) // current_prices[i, 0]))
                if shares_to_sell > 0:
                    self.positions[t] -= shares_to_sell
                    self.balance += shares_to_sell * current_prices[i, 0]

        self.current_step += 1

        if self.current_step >= len(self.data) - 1:
            self.done = True

        obs = self._next_observation()
        reward = self.balance - self.initial_balance
        done = self.done
        info = {}

        return obs, reward, done, info

    def _next_observation(self):
        price_data = []
        for t in self.tickers:
            data_t = self.data[self.data['Ticker'] == t].iloc[self.current_step - self.window_size:self.current_step]
            if len(data_t) < self.window_size:
                data_t = pd.concat([self.data[self.data['Ticker'] == t].iloc[:self.current_step], data_t], axis=0)
            price_data_t = data_t[['Close']].values
            price_data.append(price_data_t)

        # Reshape normalized price data into flattened observation vector
        obs = np.hstack(price_data).flatten()

        # Replace any NaN values with zeros
        obs = np.nan_to_num(obs)

        # Estimate the underlying asset price dynamics using the Kalman filter for each ticker
        smoothed_prices = np.zeros((self.window_size, 5*len(self.tickers)))
        for i, t in enumerate(self.tickers):
            price_data_t = price_data[i]
            kf = KalmanFilter(initial_state_mean=price_data_t[0], n_dim_obs=1)
            smoothed_prices_t, _ = kf.smooth(price_data_t.flatten())
            smoothed_prices[:,5*i:5*(i+1)] = smoothed_prices_t.reshape(self.window_size, 1)

        # Isolate cyclic components using Fourier-based spectral estimation for each ticker
        cyclic_components = np.zeros(self.window_size)
        for i, t in enumerate(self.tickers):
            price_data_t = price_data[i]
            freqs, psd = periodogram(price_data_t)
            significant_freqs = freqs[np.argsort(psd)[:5]]  # Top 5 frequencies with lowest power spectral density
            cyclic_components_t = np.sum([np.sin(2 * np.pi * f * np.arange(len(price_data_t))) for f in significant_freqs], axis=0)
            cyclic_components += cyclic_components_t

        # Model the underlying asset price dynamics using Geometric Brownian Motion (GBM) for each ticker
        price_dynamics = np.zeros((self.window_size, len(self.tickers)))
        for i, t in enumerate(self.tickers):
            price_data_t = price_data[i]
            returns = np.diff(price_data_t, axis=0) / price_data_t[:-1]
            mu = np.mean(returns, axis=0)
            sigma = np.std(returns, axis=0)
            dt = 1
            Z = norm.ppf(np.random.rand(self.window_size - 1))
            price_dynamics_t = price_data_t[0] * np.exp(np.cumsum((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z))
            price_dynamics[:len(price_dynamics_t),i] = price_dynamics_t

        # Create portfolio feature array
        portfolio_features = np.zeros((1, len(self.tickers)))
        for i, t in enumerate(self.tickers):
            portfolio_features[0,i] = self.positions.get(t, 0)

        # Combine all features into a single observation array
        obs = np.concatenate([obs, smoothed_prices.flatten(), cyclic_components, price_dynamics.flatten(), portfolio_features.flatten()])

        return obs.reshape(1, -1)

# Define the optimization function
def optimize_agent(trial):
    # Define hyperparameters to optimize
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
    gamma = trial.suggest_categorical("gamma", [0.9, 0.95, 0.99, 0.999])
    n_epochs = trial.suggest_int("n_epochs", 2, 10)
    n_steps = trial.suggest_int("n_steps", 16, 2048)

    # Train the agent using PPO algorithm
    model = PPO("MlpPolicy", env, learning_rate=learning_rate, gamma=gamma, n_epochs=n_epochs, n_steps=n_steps, verbose=0)
    model.learn(total_timesteps=100000)

    return evaluate_agent(model)

# Evaluate the agent
def evaluate_agent(model):
    # Test the agent's performance
    test_env = gym.make("stocks-v0", df=ohlcv_data)
    test_env = DummyVecEnv([lambda: test_env])

    obs = test_env.reset()
    done = False
    total_profits = []

    while not done:
        action, _states = model.predict(obs)
        obs, reward, done, info = test_env.step(action)
        total_profits.append(info[0]['total_profit'])

    # Calculate portfolio returns
    portfolio_returns = np.diff(total_profits) / total_profits[:-1]
    annualized_return = np.mean(portfolio_returns) * 252

    #TODO change to use the actual snp500 pct change
    # Calculate benchmark returns (S&P 500 Index)
    benchmark_returns = ohlcv_data['Close'].pct_change().dropna().values

    return annualized_return, calculate_performance_metrics(portfolio_returns, benchmark_returns)

def calculate_performance_metrics(portfolio_returns, benchmark_returns, risk_free_rate=0.02, sortino_target=0.0):
    # Annualize returns
    annualized_return = np.mean(portfolio_returns) * 252
    benchmark_annualized_return = np.mean(benchmark_returns) * 252

    # Max drawdown
    cum_returns = (1 + portfolio_returns).cumprod()
    running_max = np.maximum.accumulate(cum_returns)
    drawdown = (cum_returns / running_max) - 1
    max_drawdown = np.min(drawdown)

    # Sharpe ratio
    sharpe_ratio = (annualized_return - risk_free_rate) / (np.std(portfolio_returns) * np.sqrt(252))
    benchmark_sharpe_ratio = (benchmark_annualized_return - risk_free_rate) / (np.std(benchmark_returns) * np.sqrt(252))

    # Sortino ratio
    downside_returns = portfolio_returns.copy()
    downside_returns[downside_returns > sortino_target] = 0
    sortino_ratio = (annualized_return - risk_free_rate) / (np.std(downside_returns) * np.sqrt(252))

    # Performance relative to the S&P 500 Index on a risk-adjusted basis
    risk_adjusted_performance = sharpe_ratio / benchmark_sharpe_ratio

    return {
        'annualized_return': annualized_return,
        'max_drawdown': max_drawdown,
        'sharpe_ratio': sharpe_ratio,
        'benchmark_sharpe_ratio': benchmark_sharpe_ratio,
        'sortino_ratio': sortino_ratio,
        'risk_adjusted_performance': risk_adjusted_performance
    }


In [ ]:
OPTIMIZATION_ENABLED = False  # Set this to False to disable the optimization process

# Load data for the environment
ohlcv_data = pd.read_csv("sp500_ohlcv_data.csv", index_col=0, parse_dates=True)

# Initialize the environment
env = DummyVecEnv([lambda: StockTradingEnv(ohlcv_data) for _ in range(1)])

# Run the optimization if enabled
if OPTIMIZATION_ENABLED:
    study = optuna.create_study(direction="maximize")
    study.optimize(optimize_agent, n_trials=100)
    best_params = study.best_params
    print("Best hyperparameters:", best_params)
else:
    best_params = {}

# Train the PPO agent with the best hyperparameters
best_model = PPO("MlpPolicy", env, **best_params, verbose=1)
best_model.learn(total_timesteps=1000)

# Calculate and print performance metrics
annualized_return, metrics = evaluate_agent(best_model)
print("Annualized return:", annualized_return)
print(metrics)